## Multi-Agent System with LangGraph

This project demonstrates a multi-agent system using [LangGraph](https://langchain-ai.github.io/langgraph/) and free LLMs from HuggingFace.

In [1]:
!pip install -U langgraph langchain langchain-community langchain-experimental transformers google-search-results python-dotenv



In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

from langgraph.graph import StateGraph, END
from langchain_community.utilities.serpapi import SerpAPIWrapper
from langchain_experimental.tools.python.tool import PythonREPLTool
from transformers import pipeline
from langchain.llms import HuggingFacePipeline


In [3]:
# Setup LLM
hf_pipe = pipeline("text2text-generation", model="google/flan-t5-base", max_length=512)
llm = HuggingFacePipeline(pipeline=hf_pipe)

# Setup Tools
search = SerpAPIWrapper(serpapi_api_key="2726dac8f18d88e1e13e03302302f03a449c2a51aff58b7c368a75046d5d288a")
math_tool = PythonREPLTool()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu
/tmp/ipython-input-3-1299937415.py:3: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=hf_pi

In [4]:
from typing import TypedDict

class WorkflowState(TypedDict):
    input: str
    research: str
    analysis: str
    output: str


### 👥 Agents:
- 🔍 Research Agent – uses SerpAPI to search the web
- 📊 Analyst Agent – uses Python REPL + HuggingFace for logic
- 📝 Writer Agent – summarizes insights into a report

In [5]:
# === Agent 1: Research Agent ===
def research_agent(state):
    query = state['input']
    print("🔍 RESEARCHING...")
    result = search.run(query)
    return {**state, "research": result}

In [6]:
# === Agent 2: Analyst Agent ===
def analyst_agent(state):
    research = state['research']
    print("🧮 ANALYZING...")
    analysis_prompt = f"Based on the research: {research}\nSummarize key economic and environmental impacts."
    result = llm.invoke(analysis_prompt)
    return {**state, "analysis": result}

In [7]:
# === Agent 3: Writer Agent ===
def writer_agent(state):
    print("📝 WRITING FINAL REPORT...")
    content = f"Research: {state['research']}\nAnalysis: {state['analysis']}\nWrite a final short report."
    report = llm.invoke(content)
    return {**state, "output": report}

In [8]:
# === Build the LangGraph ===
builder = StateGraph(WorkflowState)
builder.add_node("research", research_agent)
builder.add_node("analysis", analyst_agent)
builder.add_node("writer", writer_agent)

builder.set_entry_point("research")
builder.add_edge("research", "analysis")
builder.add_edge("analysis", "writer")
builder.add_edge("writer", END)

In [9]:
graph = builder.compile()

# === RUN ===
result = graph.invoke({"input": "What will be the impact of electric vehicles in India by 2030?"})
print("\n✅ FINAL OUTPUT:\n", result['output'])

🔍 RESEARCHING...
🧮 ANALYZING...


Token indices sequence length is longer than the specified maximum sequence length for this model (907 > 512). Running this sequence through the model will result in indexing errors


📝 WRITING FINAL REPORT...

✅ FINAL OUTPUT:
 India's EV industry will require 1,300 acres to set up 110 GWh battery manufacturing capacity by 2030. The country's EV industry will require 1,300 acres to set up 110 GWh battery manufacturing capacity by 2030. The country's EV industry will require 1,300 acres to set up 110 GWh battery manufacturing capacity by 2030. The country's EV industry will require 1,300 acres to set up 110 GWh battery manufacturing capacity by 2030. The country's EV industry will require 1,300 acres to set up 110 GWh battery manufacturing capacity by 2030. The country's EV industry will require 1,300 acres to set up 110 GWh battery manufacturing capacity by 2030. The country's EV industry will require 1,300 acres to set up 110 GWh battery manufacturing capacity by 2030. The country's EV industry will require 1,300 acres to set up 110 GWh battery manufacturing capacity by 2030. The country's EV industry will require 1,300 acres to set up 110 GWh battery manufacturing